## Smith-Waterman algorithm

The Smith-Waterman algorithm is a variant of the Needleman-Wunsch algorithm that solves the local alignment of two sequences applying dynamic programming.

In [16]:
import numpy as np
# Load BLOSUM62 matrix
from Bio.Align import substitution_matrices
blosum62 = substitution_matrices.load("BLOSUM62")

## Needleman-Wunsch algorithm implementation

Implementation of Needleman-Wunsch algorithm for global alignment.

In [17]:
def getGlobalTables(s,t,m,gap_penalty):
    """It gets as input two sequences to align (s, t), the substitution matrix (m), and the gap penalty (gap_penalty). 
    It returns the dynamic matrix and the pointers matrix."""
    dynamic_matrix, pointers_matrix = [],[]
    # Fill both matrices with zeros
    for e in range(0,len(s)+1):
        dynamic_matrix.append([0]*(len(t)+1))
        pointers_matrix.append([0]*(len(t)+1))
        
    # Fill first row and first column
    for i in range(1,len(s)+1):
        dynamic_matrix[i][0] = dynamic_matrix[i-1][0] + gap_penalty
        pointers_matrix[i][0] = 2
        
    for j in range(1,len(t)+1):
        dynamic_matrix[0][j] = dynamic_matrix[0][j-1] + gap_penalty
        pointers_matrix[0][j] = 3
        
    # Use a nested loop to fill the rest of the matrices
    for i in range(1,len(s)+1):
        for j in range(1,len(t)+1):
            # Store the maximum
            dynamic_matrix[i][j] = max(dynamic_matrix[i-1][j-1]+m[s[i-1],t[j-1]],
                                       dynamic_matrix[i-1][j]+gap_penalty,
                                       dynamic_matrix[i][j-1]+gap_penalty)
            # Store the index of the maximum + 1
            pointers_matrix[i][j] = np.argmax([dynamic_matrix[i-1][j-1]+m[s[i-1],t[j-1]],
                                       dynamic_matrix[i-1][j]+gap_penalty,
                                       dynamic_matrix[i][j-1]+gap_penalty]) + 1
            
    return (dynamic_matrix,pointers_matrix)


In [18]:
def getGlobalTrace(p):
    """It gets as input the pointers matrix (p) and returns the trace"""

    i,j,trace = len(p)-1, len(p[0])-1, []
    
    # Trace the pointers matrix
    while p[i][j] != 0:
        trace.append(p[i][j])
        if p[i][j] == 1:
            i -= 1
            j -= 1
        elif p[i][j] == 2:
            i -= 1
        else:
            j -= 1
    # Reverse the trace
    trace.reverse()
    return trace

In [19]:
def globalAlignment(s,t,m,gap_penalty):
    """It gets as input two sequences to align (s, t), the trace, and the gap penalty. It returns the obtained alignment and its score."""
    # Define the variables: i, j, a, b
    dynamic_matrix = getGlobalTables(s,t,m,gap_penalty)[0]
    pointers_matrix = getGlobalTables(s,t,m,gap_penalty)[1]
    trace = getGlobalTrace(pointers_matrix)

    # Store the score of the alignment
    score = dynamic_matrix[len(s)][len(t)]

    i,j,sp,tp = 0,0,"",""
    # Iterate over trace
    for k in range(0,len(trace)):
        if trace[k] == 1:
            sp += s[i]
            tp += t[j]
            i += 1
            j += 1
        elif trace[k] == 2:
            sp += s[i]
            tp += '-'
            i += 1
        else:
            sp += '-'
            tp += t[j]
            j += 1
       
    return (sp,tp,score)



In [20]:
globalAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

('VIVALA-VIDA', 'VIVALADAVIS', 21.0)

## Smith-Waterman algorithm implementation

Implementation of the Smith-Waterman algorithm for local alignment

In [21]:
def getLocalTables(s,t,m,gap_penalty):
    """It gets as input the sequences to align (s, t), the substitution matrix (m), and the gap penalty (gap_penalty). 
    It returns the dynamic puntuation matrix and the decision matrix."""

    punt,dec = [],[]
    # Fill matrices with zeros
    for e in range(0,len(s)+1):
        punt.append([0]*(len(t)+1))
        dec.append([0]*(len(t)+1))
    
    # Fill the first row and column.
    for i in range(1,len(s)+1):
        dec[i][0] = 4
    for j in range(1,len(t)+1):
        dec[0][j] = 4
    
    # Fill the rest of the matrices with a nested loop
    for i in range(1,len(s)+1):
        for j in range(1,len(t)+1):
            # Store the maximum value
            punt[i][j] = max(punt[i-1][j-1]+m[s[i-1],t[j-1]],
                            punt[i-1][j] + gap_penalty,
                            punt[i][j-1] + gap_penalty,
                            0)
            # Store the index of the maximum + 1
            dec[i][j] = np.argmax([punt[i-1][j-1]+m[s[i-1],t[j-1]],
                            punt[i-1][j] + gap_penalty,
                            punt[i][j-1] + gap_penalty,
                            0]) + 1
    
    return (punt,dec)


In [22]:
getLocalTables('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 4.0, 3.0, 4.0, 0.0, 1.0, 0.0, 0, 0.0, 4.0, 3.0, 0],
  [0, 3.0, 8.0, 6.0, 3.0, 2.0, 0.0, 0, 0, 3.0, 8.0, 4.0],
  [0, 4.0, 6.0, 12.0, 8.0, 4.0, 2.0, 0, 0.0, 4.0, 6.0, 6.0],
  [0, 0.0, 3.0, 8.0, 16.0, 12.0, 8.0, 4.0, 4.0, 0.0, 3.0, 7.0],
  [0, 1.0, 2.0, 4.0, 12.0, 20.0, 16.0, 12.0, 8.0, 5.0, 2.0, 3.0],
  [0, 0.0, 0.0, 2.0, 8.0, 16.0, 24.0, 20.0, 16.0, 12.0, 8.0, 4.0],
  [0, 4.0, 3.0, 4.0, 4.0, 12.0, 20.0, 21.0, 20.0, 20.0, 16.0, 12.0],
  [0, 3.0, 8.0, 6.0, 3.0, 8.0, 16.0, 17.0, 20.0, 23.0, 24.0, 20.0],
  [0, 0, 4.0, 5.0, 4.0, 4.0, 12.0, 22.0, 18.0, 19.0, 20.0, 24.0],
  [0, 0.0, 0.0, 4.0, 9.0, 5.0, 8.0, 18.0, 26.0, 22.0, 18.0, 21.0]],
 [[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
  [4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 4],
  [4, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 3],
  [4, 1, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1],
  [4, 1, 1, 2, 1, 3, 1, 3, 1, 1, 1, 1],
  [4, 1, 1, 1, 2, 1, 3, 3, 3, 1, 1, 2],
  [4, 1, 1, 1, 1, 2, 1, 3, 1, 3, 3, 3],
  [4, 1, 1, 1, 2, 2, 2, 1, 1, 1, 3,

In [23]:
def getLocalTrace(punt,dec):
    """It gets as input the dynamic puntuation matrix and the decision matrix and it returns the trace."""

    # Start from the position with maximum score
    punt_arr=np.array(punt)
    pos_max = np.where(punt_arr == punt_arr.max())
    i=int(pos_max[0])
    j=int(pos_max[1])
    trace=[]
    while dec[i][j]!=0:
        trace.append(dec[i][j])
        if dec[i][j]==1:
            i-=1
            j-=1
        elif dec[i][j]==2:
            i-=1
        elif dec[i][j]==3:
            j-=1
    # Reverse the trace
    trace.reverse()
    return trace


In [24]:
getLocalTrace(*getLocalTables('VIVALAVIDA','VIVALADAVIS',blosum62,-4))

[1, 1, 1, 1, 1, 1, 2, 2, 1, 1]

In [25]:
def localAlignment(s,t,m,gap_penalty):
    """It gets as input the sequences to align (s, t), the substitution matrix (m), and the gap penalty (gap_penalty). 
    It returns the two aligned sequences (sp, tp), and the score of the alignment obtained (score)."""
    punt,dec=getLocalTables(s,t,m,gap_penalty)
    punt_arr = np.array(punt)
    score = punt_arr.max()
    trace=getLocalTrace(punt,dec)
    i, j, sp, tp = 0, 0, "", ""
    for k in range(0,len(trace)):
        if trace[k]==1:
            sp += s[i]
            tp += t[j]
            i += 1
            j += 1
        elif trace[k]==2:
            sp += s[i]
            tp += '-'
            i += 1
        elif trace[k]==3:
            sp += '-'
            tp += t[j]
            j += 1
            
    return (sp,tp,score)


In [26]:
localAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)

('VIVALAVIDA', 'VIVALA--DA', 26.0)

In [27]:
def formatAlignment(alignment):
    """It gets as input a tuple representing the alignment and prints this alignment in a aesthetic way."""
    s,t,score=alignment
    align=""
    # Iterate over the two sequences
    for x in zip(s,t):
        
        if x[0]==x[1]:
            align+="|"
            
        elif x[0]=="-" or x[1]=="-":
            align+=" "
            
        else:
            align+="·"

    pscore="Score: "+ str(score)
    print(s,align,t,pscore, sep="\n")

## Comparison of global and local alignment

In [28]:
alignment = globalAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)
formatAlignment(alignment)

VIVALA-VIDA
|||||| ····
VIVALADAVIS
Score: 21.0


In [29]:
alignment = localAlignment('VIVALAVIDA','VIVALADAVIS',blosum62,-4)
formatAlignment(alignment)

VIVALAVIDA
||||||  ||
VIVALA--DA
Score: 26.0
